# Importing dataset

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
sb.set()
from sklearn import metrics
import missingno as msn
%matplotlib inline
import matplotlib.pyplot as plt
reg_data = pd.read_excel('WHR2018Chapter2OnlineData.xls',sheet_name = 'SupportingFactors')
data = pd.read_excel('WHR2018Chapter2OnlineData.xls',sheet_name = 'Table2.1')
region = pd.DataFrame(reg_data[["country","Region indicator"]])  # extracting region information from supporting factors sheet

data_M = data.merge(region, on='country', how='left')         # inserting a column of region corresponding to countries to the dataset
col = list(data_M)
col.insert(1,col.pop(col.index('Region indicator')))
data_M = data_M.loc[:,col]
data_M.head(100)    # merged data

In [ ]:
data_M['Region indicator'].fillna("None", inplace=True)
print("number of regions:",len(data_M['Region indicator'].unique())-1)    # len()-1 because "none" is not a region
data_M['Region indicator'].value_counts()    # information about the regions

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(32, 5))
sb.countplot(data_M["Region indicator"])

# Visualisation of missing data

In [ ]:
data_M.info()

In [ ]:
msn.matrix(data_M.sample(1562))   # visualise the locations where the values are missing

In [ ]:
msn.bar(data_M.sample(1562))    # counting the data points present for each variable in the dataset

# Filling in the missing data

In [ ]:
import scipy as sp

In [ ]:
data_pred = data_M.interpolate(method = 'linear')    # use scipy.interpolation to fill in the missing data values
data_pred.head(1562)

In [ ]:
data_pred.info()

In [ ]:
msn.matrix(data_pred.sample(1562))    # some data points are still missing beacause extrapolation was not done and those points lie outside the range of given data

In [ ]:
msn.bar(data_pred.sample(1562))

# Visualisation of filled-in data

In [ ]:
sb.pairplot(data = data_M.drop(['year'],axis=1))    #distribution of original data

In [ ]:
sb.pairplot(data = data_pred.drop(['year'],axis=1))    #distribution of filled in data, which should resemble that of the original data

In [ ]:
data_complete = data_pred.dropna()    # we did not use extrapolation to find the missing data points outside the range of given data points because 1.their numbers are not great and 2.their values would be more unreliable as they will be predicted from predicted values.
data_complete.head(1562)

In [ ]:
data_complete.info()

In [ ]:
data_c = data_complete.drop(['year'],axis=1)
f, axes = plt.subplots(1, 1, figsize=(12, 8))
sb.heatmap(data_c.corr(), vmin = -1, vmax = 1, annot = True, fmt = ".2f")    # look at the first column or first row to see which variable has the highest correlation coefficient with Life Ladder

# Building regression models

## <font color='grey'>Preparation of train-test datasets</font>

In [ ]:
data_comp = data_complete.drop(['GINI index (World Bank estimate)'],axis = 1)

<font color='red'>**_we are not going to use this column in building our model because the number data points given in the dataset was less than half of the number of the whole dataset and our model would be entirely based on our predicted values when we filled in the missing data points_**</font>

In [ ]:
data_comp = data_comp.drop(['Standard deviation of ladder by country-year','Standard deviation/Mean of ladder by country-year'],axis = 1)

<font color='red'>**_we are not going to use these columns because they are directly related/calculated from life ladder, which is the variable that we are trying to predict_**</font>

In [ ]:
data_comp1 = data_comp.drop(['year','country','Region indicator'],axis = 1).reset_index()

In [ ]:
data_comp = data_comp1.drop(['index'],axis = 1)

<font color='red'>**_we are not going to use these columns because they are string variables/do not have a linear relationship with life ladder_**</font>

In [ ]:
data_comp.head()

In [ ]:
y = pd.DataFrame(data_comp['Life Ladder'])
X = data_comp.drop(['Life Ladder'],axis = 1)    # drop life ladder becasue it is the variable we are trying to predict

## <font color='yellow'>Hold-out validation</font>

In [ ]:
from sklearn.model_selection import train_test_split
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X, y, test_size = 0.2)

print("X_Train Set :", X_train_h.shape)
print("X_Test Set  :", X_test_h.shape)
print("y_Train Set :", y_train_h.shape)
print("y_Test Set  :", y_test_h.shape)

## <font color='yellow'>Cross-validation (K-fold)</font>

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=6)
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf)

In [ ]:
for train_index, test_index in kf.split(X,y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
    y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]

## <font color='grey'>Linear regression model</font>

In [ ]:
from sklearn.linear_model import LinearRegression
linreg_h = LinearRegression()
linreg_cv = LinearRegression()

### <font color='purple'> Using hold-out validation</font>

In [ ]:
linreg_h.fit(X_train_h, y_train_h)
print('Intercept of Regression \t: b = ', linreg_h.intercept_)
print('Coefficients of Regression \t: a = ', linreg_h.coef_)
print()
pd.DataFrame(list(zip(X_train_h.columns, linreg_h.coef_[0])), columns = ["Predictors", "Coefficients"])

In [ ]:
y_train_h_pred_L = linreg_h.predict(X_train_h)
y_test_h_pred_L = linreg_h.predict(X_test_h)

f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train_h, y_train_h_pred_L, color = "blue")
axes[0].plot(y_train_h, y_train_h, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test_h, y_test_h_pred_L, color = "green")
axes[1].plot(y_test_h, y_test_h, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
print("Goodness of Fit of Model \tTrain Dataset")
print("Score of model (R^2) \t:", linreg_h.score(X_train_h, y_train_h))
print("Error of prediction (MSE) \t:", mean_squared_error(y_train_h, y_train_h_pred_L))
print("Accuracy of prediction \t:", metrics.r2_score(y_train_h, y_train_h_pred_L))
print()
print("Goodness of Fit of Model \tTest Dataset")
print("Score of model (R^2) \t:", linreg_h.score(X_test_h, y_test_h))
print("Erroe of prediction (MSE) \t:", mean_squared_error(y_test_h, y_test_h_pred_L))
print("Accuracy of prediction \t:", metrics.r2_score(y_test_h, y_test_h_pred_L))
print()

### <font color='purple'> Using cross validation</font>

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [ ]:
linreg_cv.fit(X_train_cv, y_train_cv)
print('Intercept of Regression \t: b = ', linreg_cv.intercept_)
print('Coefficients of Regression \t: a = ', linreg_cv.coef_)
print()
pd.DataFrame(list(zip(X_train_cv.columns, linreg_cv.coef_[0])), columns = ["Predictors", "Coefficients"])

In [ ]:
y_train_cv_pred_L = cross_val_predict(linreg_cv, X_train_cv, y_train_cv, cv=6)
y_test_cv_pred_L = cross_val_predict(linreg_cv, X_test_cv, y_test_cv, cv=6)

f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train_cv, y_train_cv_pred_L, color = "blue")
axes[0].plot(y_train_cv, y_train_cv, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test_cv, y_test_cv_pred_L, color = "green")
axes[1].plot(y_test_cv, y_test_cv, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

In [ ]:
print("Goodness of Fit of Model \tTrain Dataset")
print("Score of model (R^2) \t:", cross_val_score(linreg_cv, X_train_cv, y_train_cv, cv=6))
print("Error of prediction (MSE) \t:", mean_squared_error(y_train_cv, y_train_cv_pred_L))
print("Accuracy of prediction \t:", metrics.r2_score(y_train_cv, y_train_cv_pred_L))
print()
print("Goodness of Fit of Model \tTest Dataset")
print("Score of model (R^2) \t:", cross_val_score(linreg_cv, X_test_cv, y_test_cv, cv=6))
print("Erroe of prediction (MSE) \t:", mean_squared_error(y_test_cv, y_test_cv_pred_L))
print("Accuracy of prediction \t:", metrics.r2_score(y_test_cv, y_test_cv_pred_L))
print()

## <font color='grey'>Random forest regressor model</font>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr_h = RandomForestRegressor()
rfr_cv = RandomForestRegressor()

### <font color='purple'> Using hold-out validation</font>

In [ ]:
rfr_h.fit(X_train_h,y_train_h.values.ravel())

In [ ]:
y_train_h_pred_R = rfr_h.predict(X_train_h)
y_test_h_pred_R = rfr_h.predict(X_test_h)

f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train_h, y_train_h_pred_R, color = "blue")
axes[0].plot(y_train_h, y_train_h, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test_h, y_test_h_pred_R, color = "green")
axes[1].plot(y_test_h, y_test_h, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

In [ ]:
print("Goodness of Fit of Model \tTrain Dataset")
print("Score of the model (R^2) \t:", rfr_h.score(X_train_h, y_train_h))
print("Error of prediction (MSE) \t:", mean_squared_error(y_train_h, y_train_h_pred_R))
print("Accuracy of prediction:", metrics.r2_score(y_train_h, y_train_h_pred_R))
print()
print("Goodness of Fit of Model \tTest Dataset")
print("Score of the model (R^2) \t:", rfr_h.score(X_test_h, y_test_h))
print("Error of prediction (MSE) \t:", mean_squared_error(y_test_h, y_test_h_pred_R))
print("Accuracy of prediction:", metrics.r2_score(y_test_h, y_test_h_pred_R))
print()

### <font color='purple'> Using cross validation</font>

In [ ]:
rfr_cv.fit(X_train_cv,y_train_cv.values.ravel())

In [ ]:
y_train_cv_pred_R = cross_val_predict(rfr_cv, X_train_cv, y_train_cv.values.ravel(), cv=6)
y_test_cv_pred_R = cross_val_predict(rfr_cv, X_test_cv, y_test_cv.values.ravel(), cv=6)

f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train_cv, y_train_cv_pred_R, color = "blue")
axes[0].plot(y_train_cv, y_train_cv, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test_cv, y_test_cv_pred_R, color = "green")
axes[1].plot(y_test_cv, y_test_cv, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()

In [ ]:
print("Goodness of Fit of Model \tTrain Dataset")
print("Score of model (R^2) \t:", cross_val_score(rfr_cv, X_train_cv, y_train_cv.values.ravel(), cv=6))
print("Error of prediction (MSE) \t:", mean_squared_error(y_train_cv, y_train_cv_pred_R))
print("Accuracy of prediction \t:", metrics.r2_score(y_train_cv, y_train_cv_pred_R))
print()
print("Goodness of Fit of Model \tTest Dataset")
print("Score of model (R^2) \t:", cross_val_score(rfr_cv, X_test_cv, y_test_cv.values.ravel(), cv=6))
print("Erroe of prediction (MSE) \t:", mean_squared_error(y_test_cv, y_test_cv_pred_R))
print("Accuracy of prediction \t:", metrics.r2_score(y_test_cv, y_test_cv_pred_R))
print()

# Prediction of 2019 Life Ladder

In [ ]:
data_2019 = pd.read_excel('WHR2019Chapter2OnlineData.xls',sheet_name = 'Table2.1')

In [ ]:
data_2019.info()

In [ ]:
overall = pd.DataFrame(data_2019[['Year','Country name','Life Ladder','Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity','Perceptions of corruption','Positive affect','Negative affect','Confidence in national government','Democratic Quality','Delivery Quality','Standard deviation of ladder by country-year','Standard deviation/Mean of ladder by country-year','GINI index (World Bank estimate)','GINI index (World Bank estimate), average 2000-16','gini of household income reported in Gallup, by wp5-year']])

In [ ]:
#overall.head()

In [ ]:
overall.info()

In [ ]:
msn.matrix(overall.sample(1704))
msn.bar(overall.sample(1704))

In [ ]:
pred_2019 = overall.interpolate(method = 'linear')

In [ ]:
msn.matrix(pred_2019.sample(1704))
msn.bar(pred_2019.sample(1704))

In [ ]:
pred_2019 = pred_2019.dropna()
pred_2019 = pred_2019.drop(['GINI index (World Bank estimate)'],axis = 1)
pred_2019 = pred_2019.drop(['Standard deviation of ladder by country-year','Standard deviation/Mean of ladder by country-year'],axis = 1)
pred_2019_1 = pred_2019.drop(['Year','Country name'],axis = 1).reset_index()
pred_2019 = pred_2019_1.drop(['index'],axis = 1)
y_2019 = pd.DataFrame(pred_2019['Life Ladder'])
X_2019 = pred_2019.drop(['Life Ladder'],axis = 1)  

In [ ]:
pred_2019.info()

In [ ]:
y_2019_h_pred_L = linreg_h.predict(X_2019)
y_2019_cv_pred_L = cross_val_predict(linreg_cv, X_2019, y_2019, cv=6)
y_2019_h_pred_R = rfr_h.predict(X_2019)
y_2019_cv_pred_R = cross_val_predict(rfr_cv, X_2019, y_2019.values.ravel(), cv=6)

In [ ]:
f, axes = plt.subplots(1, 4, figsize=(24, 6))
axes[0].scatter(y_2019, y_2019_h_pred_L, color = "blue")
axes[0].plot(y_2019, y_2019, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable")
axes[0].set_ylabel("Predicted values of the Response Variable")

axes[1].scatter(y_2019, y_2019_cv_pred_L, color = "green")
axes[1].plot(y_2019, y_2019, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable")
axes[1].set_ylabel("Predicted values of the Response Variable")

axes[2].scatter(y_2019, y_2019_h_pred_R, color = "green")
axes[2].plot(y_2019, y_2019, 'w-', linewidth = 1)
axes[2].set_xlabel("True values of the Response Variable")
axes[2].set_ylabel("Predicted values of the Response Variable")

axes[3].scatter(y_2019, y_2019_cv_pred_R, color = "green")
axes[3].plot(y_2019, y_2019, 'w-', linewidth = 1)
axes[3].set_xlabel("True values of the Response Variable")
axes[3].set_ylabel("Predicted values of the Response Variable")
plt.show()

In [ ]:
print("Goodness of Fit of Model 1 \t")    # model 1 is using the linear regression model trained with hold-out validation
print("Score of the model (R^2) \t:", linreg_h.score(X_train_h, y_train_h))
print("Error of prediction (MSE) \t:", mean_squared_error(y_2019, y_2019_h_pred_L))
print("Accuracy of prediction:", metrics.r2_score(y_2019, y_2019_h_pred_L))
print()

print("Goodness of Fit of Model 2 \t")    # model 2 is using the linear regression model trained with cross validation
print("Score of model (R^2) \t:", cross_val_score(linreg_cv, X_train_cv, y_train_cv, cv=6))
print("Erroe of prediction (MSE) \t:", mean_squared_error(y_2019, y_2019_cv_pred_L))
print("Accuracy of prediction \t:", metrics.r2_score(y_2019, y_2019_cv_pred_L))
print()

print("Goodness of Fit of Model 3 \t")    # model 3 is using the random forest regressor model trained with hold-out validation
print("Score of the model (R^2) \t:", rfr_h.score(X_train_h, y_train_h.values.ravel()))
print("Error of prediction (MSE) \t:", mean_squared_error(y_2019, y_2019_h_pred_R))
print("Accuracy of prediction:", metrics.r2_score(y_2019, y_2019_h_pred_R))
print()

print("Goodness of Fit of Model 4 \t")    # model 4 is using the random forest regressor model trained with cross validation
print("Score of model (R^2) \t:", cross_val_score(rfr_cv, X_train_cv, y_train_cv.values.ravel(), cv=6))
print("Error of prediction (MSE) \t:", mean_squared_error(y_2019, y_2019_cv_pred_R))
print("Accuracy of prediction \t:", metrics.r2_score(y_2019, y_2019_cv_pred_R))
print()